In [ ]:
library(tidyverse)
source("scripts/r/functions_objects.R")

In [ ]:
urban_mafs <- read_delim(snakemake@input[["urban_mafs"]], delim = "\t") %>% 
    rename("chrom" = "chromo", "pos" = "position", "urban_maf" = "knownEM") %>% 
    dplyr::select(chrom, pos, urban_maf)
rural_mafs <- read_delim(snakemake@input[["rural_mafs"]], delim = "\t") %>% 
    rename("chrom" = "chromo", "pos" = "position", "rural_maf" = "knownEM") %>% 
    dplyr::select(chrom, pos, rural_maf)

In [ ]:
load_cont_output <- function(path){
    base <- basename(path)
    split <- str_extract(base, pattern = "(?<=split).*(?=_summary)")
    
    df <- suppressMessages(read.table(path, header = TRUE)) %>% 
        rename("C2_log10_pval" = "log10.1.pval.") %>% 
        mutate(C2_pval = 10^(-1 * C2_log10_pval)) %>% 
        dplyr::select(C2_std, C2_log10_pval, C2_pval) %>% 
        mutate(split = split)
    return(df)
}

site_order <- suppressMessages(read_delim(snakemake@input[["site_order"]], 
                                          delim = "\t",
                                          col_names = c("chrom", "pos")))

c2_df <- snakemake@input[["cont_out"]] %>% 
    purrr::map_dfr(., load_cont_output) %>%
    bind_cols(., site_order) %>% 
    arrange(chrom, pos) %>%
    dplyr::select(chrom, pos, C2_pval)

In [ ]:
win_size <- snakemake@params[["win_size"]]
all_dat <- urban_mafs %>% 
    left_join(rural_mafs, by = c("chrom", "pos")) %>% 
    mutate(mean_maf = (urban_maf + rural_maf) / 2) %>% 
    mutate(mean_maf = ifelse(mean_maf > 0.5, 1 - mean_maf, mean_maf)) %>% 
    dplyr::select(chrom, pos, mean_maf) %>%
    left_join(., c2_df, by = c("chrom", "pos")) %>% 
    mutate(win = paste0(chrom, ":", ceiling(pos / win_size) * win_size - win_size + 1, "-", ceiling(pos / win_size) * win_size))

In [ ]:
write_delim(all_dat, snakemake@output[["wza_input"]], delim = "\t")